In [1]:
import sagemaker
import boto3
from sagemaker.huggingface.model import HuggingFacePredictor
import json

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/xdg-ubuntu/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/willem/.config/sagemaker/config.yaml


In [2]:
# AWS login to use sagemaker endpoints
session = boto3.Session(profile_name='vrt-analytics-engineer-nonsensitive')
sagemaker_session = sagemaker.Session(boto_session=session)
role = sagemaker.get_execution_role(sagemaker_session=sagemaker_session)

# Vector store endpoint
endpoint_name = "faiss-endpoint-1722716142"
faiss_vector_store = sagemaker.predictor.Predictor(endpoint_name,sagemaker_session=sagemaker_session)
assert faiss_vector_store.endpoint_context().properties['Status'] == 'InService'

# LLM endpoint
llm = HuggingFacePredictor(endpoint_name="huggingface-pytorch-tgi-inference-2024-07-30-20-23-30-977",sagemaker_session=sagemaker_session)

In [3]:
question = "iets met geld en beleggen"

In [4]:
# Fetch content
payload = json.dumps({
    "text": question,
})

out = faiss_vector_store.predict(
    payload,
    initial_args={"ContentType": "application/json", "Accept": "application/json"}
)
out = json.loads(out)

In [13]:
# Generate prompt
system_prompt = """Je bent een hulpvaardige assistent. 
Je krijgt vragen over de catalogus van programma's van het videoplatform VRT MAX. 
Je probeert mensen te helpen om programma's aan te bevelen die aansluiten bij hun vraag.
Daarvoor krijg je een beschrijving van een aantal programma's.
Aan jou om wat relevant is aan te bevelen.
"""

user_prompt = ""
for item in out.keys():
    
    user_prompt += "Het programma met de naam " + out[item]["metadata"]["mediacontent_pagetitle_program"] +\
          " heeft volgende beschrijving: " + out[item]["metadata"]["mediacontent_page_description"] 
    
    # + " en in het programma wordt er onder meer het volgende gezegd dat misschien relevant is voor de vraag: " + \out[item]["page_content"]

user_prompt += " Tot zover alle informatie over de programmas."

user_prompt += " Gelieve met die informatie een antwoord te geven op volgende vraag: " + question

In [14]:
# Ask question
# Prompt to generate
messages=[
    { "role": "system", "content": system_prompt },
    { "role": "user", "content": user_prompt }
  ]

# Generation arguments
parameters = {
    "model": "meta-llama/Meta-Llama-3-8B-Instruct", # placholder, needed
    "top_p": 0.6,
    "temperature": 0.9,
    "max_tokens": 512,
    "stop": ["<|eot_id|>"],
}

chat = llm.predict({"messages" :messages, **parameters})

print(chat["choices"][0]["message"]["content"].strip())

Based on the descriptions, I think I can help you find a relevant program.

If you're interested in learning more about investing and belegging, I would recommend "Universiteit van Vlaanderen" or "Schermtijd". Both programs seem to focus on the topic of investing and belegging, with "Universiteit van Vlaanderen" providing a more educational approach and "Schermtijd" exploring the topic in a more practical and hands-on way.

If you're looking for a program that also covers fact-checking and investigating financial scams, "Factcheckers" might be a good choice. Although it's not exclusively focused on belegging, it does touch on the topic of online financial scams and how to avoid them.

If you're looking for a more general program that provides an overview of the economic actuality, "De markt" might be a good option.

Let me know if any of these programs interest you, or if you have any other specific preferences or interests!
